# Simple ReAct Agent from Scratch

In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [ ]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [17]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [18]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4-0125-preview", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [19]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [20]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [21]:
abot = Agent(prompt)

In [22]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the dog's weight using average_dog_weight for a Toy Poodle.
Action: average_dog_weight: Toy Poodle
PAUSE


In [23]:
result = average_dog_weight("Toy Poodle")

In [24]:
result

'a toy poodles average weight is 7 lbs'

In [25]:
next_prompt = "Observation: {}".format(result)

In [26]:
abot(next_prompt)

'Answer: A Toy Poodle weighs 7 lbs.'

In [27]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [28]:
abot = Agent(prompt)

In [29]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: To find the combined weight of a Border Collie and a Scottish Terrier, I need to find the average weight of each breed and then add them together. I will start by finding the average weight of a Border Collie.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [30]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [31]:
abot(next_prompt)

'Thought: Now that I know the average weight of a Border Collie is 37 lbs, I need to find the average weight of a Scottish Terrier to calculate the combined weight.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [32]:
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [33]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [34]:
abot(next_prompt)

"Thought: The observation for the Scottish Terrier's weight seems to be missing the unit, but based on the context, it's reasonable to assume the weight is in pounds. With the Scottish Terrier's average weight being 57 lbs and the Border Collie's average weight being 37 lbs, I can now calculate their combined weight.\nAction: calculate: 37 + 57\nPAUSE"

# ADD LOOP

In [36]:
# The 'r' before the string makes it a raw string
action_re = re.compile(r'^Action: (\w+): (.*)$')

In [43]:
def query(question, max_turns=2):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [44]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: To find the combined weight of a Border Collie and a Scottish Terrier, I need to first find the average weight of each breed and then add those weights together.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I know the average weight of a Border Collie is 37 lbs, I need to find the average weight of a Scottish Terrier to calculate the combined weight.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
